<h3>Google Sheets API Credentials</h3>
<pre>Email: amberpro69@gmail.com
Password: amberdrake4283</pre>

In [18]:
import re
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [4]:
hareruyamtg_df = pd.read_csv('hareruyamtg.csv')
cardkingdom_df = pd.read_csv('cardkingdom.csv')
tcgplayer_details_df = pd.read_csv('tcgplayer_details.csv')
tcgplayer_prices_df = pd.read_csv('tcgplayer_prices.csv')

In [5]:
tcgplayer_df = pd.merge(tcgplayer_details_df,tcgplayer_prices_df,how='left')

In [6]:
hareruyamtg_df = hareruyamtg_df[~hareruyamtg_df['name'].str.contains('【Art Card】')]
hareruyamtg_df['set'] = hareruyamtg_df['name'].str.extract('\[(.+?)\]')
hareruyamtg_df['name'] = hareruyamtg_df['name'].str.extract('《(.+?)》')
hareruyamtg_df = hareruyamtg_df[~((hareruyamtg_df['name'].isna()) | (hareruyamtg_df['set'].isna()))].drop_duplicates(subset=['name','set'],ignore_index=True).reset_index(drop=True)

In [7]:
cardkingdom_df['set'] = cardkingdom_df['name'].str.split(': ',expand=True)[0]
cardkingdom_df['name'] = cardkingdom_df['name'].str.split(': ',expand=True)[1]
cardkingdom_df = cardkingdom_df[~((cardkingdom_df['name'].isna()) | (cardkingdom_df['set'].isna()))].drop_duplicates(subset=['name','set'],ignore_index=True).reset_index(drop=True)

In [8]:
tcgplayer_df.drop_duplicates(subset=['product_name','set_code'], ignore_index=True, inplace=True)

In [9]:
hareruyamtg_df = hareruyamtg_df[['name','set','stock','price']].rename(columns={'price':'hareruyamtg_price'})
cardkingdom_df = cardkingdom_df[['name','set','price']].rename(columns={'price':'cardkingdom_price'})
tcgplayer_df = tcgplayer_df[['product_name','set_code','buy_list_market_price']].rename(columns={'product_name':'name','set_code':'set','buy_list_market_price':'tcgplayer_buy_list_market_price'})

In [10]:
merged_df = pd.merge(hareruyamtg_df,cardkingdom_df,how='left',left_on=['name','set'],right_on=['name','set'])
merged_df = pd.merge(merged_df,tcgplayer_df,how='left',left_on=['name','set'],right_on=['name','set'])
merged_df = merged_df[~((merged_df['cardkingdom_price'].isna()) & (merged_df['tcgplayer_buy_list_market_price'].isna()))]
merged_df.to_csv('merged.csv',index=False)

In [26]:
credentials = ServiceAccountCredentials.from_json_keyfile_name('cards-sheet-680d878a45f2.json')
client = gspread.authorize(credentials)

In [30]:
sheet = client.open('cards')
data_sheet = sheet.worksheet('cards')

In [35]:
set_with_dataframe(data_sheet, merged_df)

In [34]:
data_sheet.clear()

{'spreadsheetId': '1D-vt-DaPAKCCq864Vz1NfS0nY0jOa7Jqb6vST5m6eoQ',
 'clearedRange': 'cards!A1:Z4478'}